In [1]:
!pip install -qU torch_snippets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.0 MB/s eta 0:00:00


In [2]:
#@ Doownloading datasets:
import os
if not os.path.exists('images'):
    %pip install -qU torch_snippets
    from google.colab import files
    files.upload() # upload kaggle.json
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !ls ~/.kaggle
    !chmod 600 /root/.kaggle/kaggle.json
    !kaggle datasets download -d sixhky/open-images-bus-trucks/
    !unzip -qq open-images-bus-trucks.zip
    !rm open-images-bus-trucks.zip

Saving kaggle.json to kaggle.json
kaggle.json
Dataset URL: https://www.kaggle.com/datasets/sixhky/open-images-bus-trucks/versions/
License(s): unknown
 99% 364M/367M [00:03<00:00, 129MB/s]
100% 367M/367M [00:03<00:00, 104MB/s]


In [3]:
from torch_snippets import *
from PIL import Image
Image_root='images/images'
DF_RAW=df=pd.read_csv('df.csv')

In [5]:
#@ Defining the indices corresponding to labels and targets:
label2target={l:t+1 for t, l in enumerate(DF_RAW['LabelName'].unique())}
label2target['background']=0
target2label={t:l for t, l in label2target.items()}
background_class=label2target['background']
num_classes=len(label2target)

In [11]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
import glob

In [10]:
def preprocess(img):
  img=torch.tensor(img).permute(2, 0, 1)
  return img.to(device).float()

In [14]:
class OpenDataset(torch.utils.data.Dataset):
  w, h= 224, 224
  def __init__(self, df, image_dir=Image_root):
    self.image_dir=image_dir
    self.files=glob.glob(self.image_dir + '/*')
    self.df=df
    self.image_infos=df.ImageID.unique()

  def __getitem__(self, ix):
    image_id=self.image_infos[ix]
    img_path=find(image_id, self.files)
    img=Image.open(img_path).convert("RGB")
    img=np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.
    data=df[df['ImageID']==image_id]
    labels=data['LabelName'].values.tolist()
    data=data[['XMin', 'YMin', 'XMax', 'YMax']].values

    #converting to absolute coordinates:
    data[:, [0, 2]]*=self.w
    data[:, [1, 3]]*=self.h
    boxes=data.astype(np.uint32).tolist()

    #torch FRCNN expects ground truth as a dictionary of tensors:
    target={}
    target['boxes']=torch.tensor(boxes).float()
    target['labels']=torch.tensor([label2target[1] for i in labels].long())
    img=preprocess(img)
    return img, target

  def __len__(self):
    return len(self.image_infos)

  def collate_func(self, batch):
    return tuple(zip(*batch))